In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, Dataset

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the LSTM-based RNN
class MusicRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2):
        super(MusicRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, hidden):
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device),
                torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device))


# Preprocessing Data: Convert MIDI-like sequence to input-output pairs
def preprocess_data(midi_data, sequence_length=50):
    unique_notes = sorted(set(midi_data))
    note_to_int = {note: i for i, note in enumerate(unique_notes)}
    int_to_note = {i: note for i, note in enumerate(unique_notes)}

    # Encode notes as integers
    encoded_notes = [note_to_int[note] for note in midi_data]

    # Create input-output sequences
    input_sequences = []
    output_sequences = []
    for i in range(len(encoded_notes) - sequence_length):
        input_sequences.append(encoded_notes[i:i + sequence_length])
        output_sequences.append(encoded_notes[i + sequence_length])

    return np.array(input_sequences), np.array(output_sequences), note_to_int, int_to_note


# Custom Dataset for MIDI Sequences
class MusicDataset(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = torch.tensor(inputs, dtype=torch.float32)
        self.outputs = torch.tensor(outputs, dtype=torch.long)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.outputs[idx]


# Generate Music from the Trained Model
def generate_music(model, start_sequence, int_to_note, num_notes=100):
    model.eval()
    generated_notes = start_sequence.tolist()  # Convert input to list if it's a tensor
    
    # Initialize hidden state for batch size 1
    hidden = model.init_hidden(batch_size=1)  # Ensure batch size matches the input

    # Prepare the input sequence by adding a batch dimension
    input_sequence = torch.tensor(start_sequence, dtype=torch.float32).unsqueeze(0).to(device)

    for _ in range(num_notes):
        output, hidden = model(input_sequence, hidden)  # Forward pass through the model
        probabilities = nn.functional.softmax(output[:, -1, :], dim=-1)
        predicted_note = torch.multinomial(probabilities, num_samples=1).item()
        
        generated_notes.append(predicted_note)
        
        # Prepare the next input sequence by appending the predicted note
        predicted_note_tensor = torch.tensor([[predicted_note]], dtype=torch.float32).to(device)
        input_sequence = torch.cat([input_sequence[:, 1:, :], predicted_note_tensor.unsqueeze(0)], dim=1)
    
    # Convert generated notes back to readable note names
    return [int_to_note[note] for note in generated_notes]



# Main Script
# Example MIDI-like data
midi_data = ["C4", "D4", "E4", "F4", "G4", "A4", "B4", "C5"] * 100  # Simplified example

# Preprocess the data
sequence_length = 50
input_sequences, output_sequences, note_to_int, int_to_note = preprocess_data(midi_data, sequence_length)

# Prepare dataset and dataloader
dataset = MusicDataset(input_sequences, output_sequences)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Model Configuration
input_size = 1
hidden_size = 128
output_size = len(note_to_int)
num_layers = 2
num_epochs = 10
learning_rate = 0.001

model = MusicRNN(input_size, hidden_size, output_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
print("Starting Training...")
for epoch in range(num_epochs):
    total_loss = 0
    model.train()

    for inputs, targets in dataloader:
        batch_size = inputs.size(0)
        hidden = model.init_hidden(batch_size)

        inputs = inputs.unsqueeze(-1).to(device)  # Add feature dimension
        targets = targets.to(device)

        optimizer.zero_grad()
        outputs, hidden = model(inputs, hidden)
        loss = criterion(outputs[:, -1, :], targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(dataloader):.4f}")

print("Training Complete.")



Starting Training...
Epoch 1/10, Loss: 2.0720
Epoch 2/10, Loss: 1.9722
Epoch 3/10, Loss: 1.2325
Epoch 4/10, Loss: 0.4891
Epoch 5/10, Loss: 0.2200
Epoch 6/10, Loss: 0.1182
Epoch 7/10, Loss: 0.0723
Epoch 8/10, Loss: 0.0498
Epoch 9/10, Loss: 0.0366
Epoch 10/10, Loss: 0.0286
Training Complete.


In [20]:
def generate_music(model, start_sequence, int_to_note, num_notes=100):
    model.eval()
    generated_notes = start_sequence.tolist()  # Convert input sequence to a list

    # Initialize hidden state for batch size 1
    batch_size = 1
    hidden = model.init_hidden(batch_size)  # Ensure hidden state matches batch size

    # Ensure input sequence is 3D (batch_size=1, sequence_length, input_size=1)
    input_sequence = torch.tensor(start_sequence, dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(device)

    for _ in range(num_notes):
        # Forward pass through the model
        output, hidden = model(input_sequence, hidden)

        # Get probabilities for the next note
        probabilities = nn.functional.softmax(output[:, -1, :], dim=-1)
        predicted_note = torch.multinomial(probabilities, num_samples=1).item()

        # Append the predicted note to the sequence
        predicted_note_tensor = torch.tensor([[predicted_note]], dtype=torch.float32).to(device).unsqueeze(-1)
        input_sequence = torch.cat([input_sequence[:, 1:, :], predicted_note_tensor], dim=1)

    # Convert generated notes to readable format
    return [int_to_note[note] for note in generated_notes]


In [21]:
# Generate music
print("Generating Music...")
start_sequence = input_sequences[0]  # Use the first sequence as the seed
generated_notes = generate_music(model, start_sequence, int_to_note, num_notes=100)
print("Generated Notes:", generated_notes)


Generating Music...
Generated Notes: ['C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4', 'C5', 'C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4', 'C5', 'C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4', 'C5', 'C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4', 'C5', 'C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4', 'C5', 'C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4', 'C5', 'C4', 'D4']


In [22]:
from midiutil import MIDIFile

def create_midi_file(notes, filename="generated_music.mid", duration=1, tempo=120):
    """
    Creates a MIDI file from a list of notes.
    :param notes: List of note names or MIDI pitches (integers).
    :param filename: Output file name for the MIDI file.
    :param duration: Duration of each note in beats.
    :param tempo: Tempo in beats per minute.
    """
    # MIDI file with one track
    midi = MIDIFile(1)
    track = 0
    time = 0  # Start at the beginning
    midi.addTempo(track, time, tempo)  # Set tempo

    for note in notes:
        # Convert note name to MIDI pitch if needed (e.g., "C4" -> 60)
        if isinstance(note, str):
            pitch = note_to_midi_pitch(note)  # Convert note name to MIDI pitch
        else:
            pitch = note  # If already a MIDI pitch
        
        # Add the note
        midi.addNote(track, channel=0, pitch=pitch, time=time, duration=duration, volume=100)
        time += duration  # Move to the next note time

    # Write the MIDI file
    with open(filename, "wb") as output_file:
        midi.writeFile(output_file)
    print(f"MIDI file saved as {filename}")

def note_to_midi_pitch(note):
    """
    Converts a note name to a MIDI pitch.
    :param note: Note name (e.g., "C4").
    :return: MIDI pitch (e.g., 60 for "C4").
    """
    note_map = {"C": 0, "D": 2, "E": 4, "F": 5, "G": 7, "A": 9, "B": 11}
    octave = int(note[-1])  # Extract octave
    key = note[:-1]  # Extract note name
    semitone = note_map[key[0]]
    if len(key) > 1:  # Handle sharps (#) and flats (b)
        if key[1] == "#":
            semitone += 1
        elif key[1] == "b":
            semitone -= 1
    return 12 * (octave + 1) + semitone

# Generate MIDI file from notes
create_midi_file(generated_notes, filename="generated_music.mid")



MIDI file saved as generated_music.mid


In [24]:
# Automatically play the MIDI file
import os
import platform

def play_midi_file(filename):
    if platform.system() == "Darwin":  # macOS
        os.system(f"open {filename}")
    elif platform.system() == "Windows":  # Windows
        os.system(f"start {filename}")
    elif platform.system() == "Linux":  # Linux
        os.system(f"xdg-open {filename}")
    else:
        print("Unable to automatically play the MIDI file. Open it manually.")

play_midi_file("generated_music.mid")
